# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 26 2022 12:00AM,238112,10,6503905,Emerginnova,Released
1,May 26 2022 12:00AM,238116,10,BPS1234,Beach Patient Assistance,Released
2,May 26 2022 12:00AM,238118,10,8057007,Therapeutics Retain 2,Released
3,May 25 2022 3:51PM,238117,16,SSF-8061506,Sartorius Stedim Filters Inc.,Released
4,May 25 2022 3:39PM,238114,21,OTM100916817,"NBTY Global, Inc.",Released
5,May 25 2022 3:38PM,238113,21,OTM100916816,"NBTY Global, Inc.",Released
6,May 25 2022 3:33PM,238111,22,615815,"NBTY Global, Inc.",Released
7,May 25 2022 3:28PM,238110,10,Enova-10005-BO,Emerginnova,Released
8,May 25 2022 3:25PM,238109,10,Enova-10015,Emerginnova,Released
9,May 25 2022 3:25PM,238109,10,Enova-10027,Emerginnova,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
35,238113,Released,1
36,238114,Released,1
37,238116,Released,1
38,238117,Released,1
39,238118,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
238113,NaN,NaN,1.0
238114,NaN,NaN,1.0
238116,NaN,NaN,1.0
238117,NaN,NaN,1.0
238118,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238021,0.0,31.0,4.0
238033,0.0,0.0,2.0
238042,0.0,1.0,0.0
238045,5.0,17.0,7.0
238050,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238021,0,31,4
238033,0,0,2
238042,0,1,0
238045,5,17,7
238050,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238021,0,31,4
1,238033,0,0,2
2,238042,0,1,0
3,238045,5,17,7
4,238050,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238021,,31,4
1,238033,,,2
2,238042,,1,
3,238045,5,17,7
4,238050,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 26 2022 12:00AM,238112,10,Emerginnova
1,May 26 2022 12:00AM,238116,10,Beach Patient Assistance
2,May 26 2022 12:00AM,238118,10,Therapeutics Retain 2
3,May 25 2022 3:51PM,238117,16,Sartorius Stedim Filters Inc.
4,May 25 2022 3:39PM,238114,21,"NBTY Global, Inc."
5,May 25 2022 3:38PM,238113,21,"NBTY Global, Inc."
6,May 25 2022 3:33PM,238111,22,"NBTY Global, Inc."
7,May 25 2022 3:28PM,238110,10,Emerginnova
8,May 25 2022 3:25PM,238109,10,Emerginnova
11,May 25 2022 3:22PM,238108,19,Eli Lilly and Company


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 26 2022 12:00AM,238112,10,Emerginnova,,,1
1,May 26 2022 12:00AM,238116,10,Beach Patient Assistance,,,1
2,May 26 2022 12:00AM,238118,10,Therapeutics Retain 2,,,1
3,May 25 2022 3:51PM,238117,16,Sartorius Stedim Filters Inc.,,,1
4,May 25 2022 3:39PM,238114,21,"NBTY Global, Inc.",,,1
5,May 25 2022 3:38PM,238113,21,"NBTY Global, Inc.",,,1
6,May 25 2022 3:33PM,238111,22,"NBTY Global, Inc.",,,1
7,May 25 2022 3:28PM,238110,10,Emerginnova,,,1
8,May 25 2022 3:25PM,238109,10,Emerginnova,,,3
9,May 25 2022 3:22PM,238108,19,Eli Lilly and Company,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 26 2022 12:00AM,238112,10,Emerginnova,1,,
1,May 26 2022 12:00AM,238116,10,Beach Patient Assistance,1,,
2,May 26 2022 12:00AM,238118,10,Therapeutics Retain 2,1,,
3,May 25 2022 3:51PM,238117,16,Sartorius Stedim Filters Inc.,1,,
4,May 25 2022 3:39PM,238114,21,"NBTY Global, Inc.",1,,
5,May 25 2022 3:38PM,238113,21,"NBTY Global, Inc.",1,,
6,May 25 2022 3:33PM,238111,22,"NBTY Global, Inc.",1,,
7,May 25 2022 3:28PM,238110,10,Emerginnova,1,,
8,May 25 2022 3:25PM,238109,10,Emerginnova,3,,
9,May 25 2022 3:22PM,238108,19,Eli Lilly and Company,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 26 2022 12:00AM,238112,10,Emerginnova,1,,
1,May 26 2022 12:00AM,238116,10,Beach Patient Assistance,1,,
2,May 26 2022 12:00AM,238118,10,Therapeutics Retain 2,1,,
3,May 25 2022 3:51PM,238117,16,Sartorius Stedim Filters Inc.,1,,
4,May 25 2022 3:39PM,238114,21,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 26 2022 12:00AM,238112,10,Emerginnova,1.0,NaN,NaN
1,May 26 2022 12:00AM,238116,10,Beach Patient Assistance,1.0,NaN,NaN
2,May 26 2022 12:00AM,238118,10,Therapeutics Retain 2,1.0,NaN,NaN
3,May 25 2022 3:51PM,238117,16,Sartorius Stedim Filters Inc.,1.0,NaN,NaN
4,May 25 2022 3:39PM,238114,21,"NBTY Global, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 26 2022 12:00AM,238112,10,Emerginnova,1.0,0.0,0.0
1,May 26 2022 12:00AM,238116,10,Beach Patient Assistance,1.0,0.0,0.0
2,May 26 2022 12:00AM,238118,10,Therapeutics Retain 2,1.0,0.0,0.0
3,May 25 2022 3:51PM,238117,16,Sartorius Stedim Filters Inc.,1.0,0.0,0.0
4,May 25 2022 3:39PM,238114,21,"NBTY Global, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3571367,46.0,26.0,5.0
16,714301,3.0,0.0,0.0
18,476137,3.0,0.0,0.0
19,714237,2.0,1.0,0.0
20,1190384,97.0,34.0,0.0
21,476227,2.0,0.0,0.0
22,952341,3.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3571367,46.0,26.0,5.0
1,16,714301,3.0,0.0,0.0
2,18,476137,3.0,0.0,0.0
3,19,714237,2.0,1.0,0.0
4,20,1190384,97.0,34.0,0.0
5,21,476227,2.0,0.0,0.0
6,22,952341,3.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,46.0,26.0,5.0
1,16,3.0,0.0,0.0
2,18,3.0,0.0,0.0
3,19,2.0,1.0,0.0
4,20,97.0,34.0,0.0
5,21,2.0,0.0,0.0
6,22,3.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,46.0
1,16,Released,3.0
2,18,Released,3.0
3,19,Released,2.0
4,20,Released,97.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,18,19,20,21,22
Status,,,,,,,
Completed,5.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,26.0,0.0,0.0,1.0,34.0,0.0,1.0
Released,46.0,3.0,3.0,2.0,97.0,2.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,18,19,20,21,22
0,Completed,5.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,26.0,0.0,0.0,1.0,34.0,0.0,1.0
2,Released,46.0,3.0,3.0,2.0,97.0,2.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,18,19,20,21,22
0,Completed,5.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,26.0,0.0,0.0,1.0,34.0,0.0,1.0
2,Released,46.0,3.0,3.0,2.0,97.0,2.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()